In [42]:
# Import Dependencies

import pandas as pd
from fbprophet import Prophet
import requests
from pprint import pprint
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs


In [2]:
# Read Bitcoin daily data from Alpha Vantage API

av_api_key = 'IYOYBP274BRSLPM0'

url = "https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=CAD&apikey={av_api_key}"

data = requests.get(url).json()
bc_data = data["Time Series (Digital Currency Daily)"]

In [3]:
# Loop through json response API data and transform into lists in python to create pandas dataframe

date = []
open_CAD = []
open_USD = []
high_CAD = []
high_USD = []
low_CAD = []
low_USD = []
close_CAD = []
close_USD = []
volume = []

date = list(bc_data.keys())

for d in date:
  value = bc_data[d]
  close_CAD.append(value['4a. close (CAD)'])
  close_USD.append(value['4b. close (USD)'])
  volume.append(value['5. volume'])
  open_CAD.append(value['1a. open (CAD)'])
  open_USD.append(value['1b. open (USD)'])
  high_CAD.append(value['2a. high (CAD)'])
  high_USD.append(value['2b. high (USD)'])
  low_CAD.append(value['3a. low (CAD)'])
  low_USD.append(value['3b. low (USD)'])

In [4]:
# Create Pandas dataframe

df = pd.DataFrame({"Date": date, "Open_CAD": open_CAD, "Open_USD": open_USD, "High_CAD": high_CAD, "High_USD": high_USD, "Low_CAD": low_CAD, "Low_USD": low_USD, "Close_CAD": close_CAD, "Close_USD":close_USD, "Volume":volume})
df.head()

,Date,Open_CAD,Open_USD,High_CAD,High_USD,Low_CAD,Low_USD,Close_CAD,Close_USD,Volume
0,2021-03-13,71355.48484000,57221.72000000,71462.07840000,57307.20000000,70222.19877000,56312.91000000,70737.49658000,56726.14000000,2339.10214500
1,2021-03-12,72043.11805000,57773.15000000,72427.64297000,58081.51000000,68538.66148000,54962.84000000,71355.48484000,57221.72000000,73405.40604700
2,2021-03-11,69646.93273000,55851.59000000,72513.05000000,58150.00000000,67678.20654000,54272.82000000,72043.13052000,57773.16000000,81914.81285900
3,2021-03-10,68428.71349000,54874.67000000,71562.44943000,57387.69000000,66097.23500000,53005.00000000,69646.93273000,55851.59000000,84749.23894300
4,2021-03-09,65311.84946000,52375.18000000,68454.06500000,54895.00000000,64581.39427000,51789.41000000,68440.97150000,54884.50000000,71656.73707600


In [5]:
# Read Bitcoin crypto rating data from Alpha Vantage API


crypto_rating_url = 'https://www.alphavantage.co/query?function=CRYPTO_RATING&symbol=BTC&apikey={av_api_key}'

rating_data = requests.get(crypto_rating_url).json()

rating = rating_data['Crypto Rating (FCAS)']['3. fcas rating']
score = rating_data['Crypto Rating (FCAS)']['4. fcas score']
refresh_date = rating_data['Crypto Rating (FCAS)']['8. last refreshed']


In [7]:
# Create a pandas df

rating_df = pd.DataFrame({"Date": refresh_date, "Rank": rating, "Score": score}, index=[0])
rating_df 

,Date,Rank,Score
0,2021-03-13 00:00:00,Superb,907


In [43]:
# Coin Market Cap URL of page to be scraped for Market Cap data

cmc_url = 'https://coinmarketcap.com/'

# Retrieve page with the requests module

response = requests.get(cmc_url)

# Create BeautifulSoup object; parse with 'html.parser'

soup = bs(response.text, 'html.parser')

In [84]:
#Scrape url for the data

results = soup.find_all('div', class_='sc-33i2yg-0 dOnegn')

for r in results:
  header = r.find_all('span', class_="sc-12ja2s9-0 eALoKW")
 
header

[<span class="sc-12ja2s9-0 eALoKW">Cryptocurrencies<!-- -->:  <a class="cmc-link" href="/all/views/all/">8,795</a></span>,
 <span class="sc-12ja2s9-0 eALoKW">Markets<!-- -->:  <a class="cmc-link" href="/currencies/volume/24-hour/">36,346</a></span>,
 <span class="sc-12ja2s9-0 eALoKW">Market Cap<!-- -->:  <a class="cmc-link" href="/charts/">$1,819,120,269,001</a></span>,
 <span class="sc-12ja2s9-0 eALoKW">24h Vol<!-- -->:  <a class="cmc-link" href="/charts/">$149,124,257,424</a></span>,
 <span class="sc-12ja2s9-0 eALoKW">Dominance<!-- -->:  <a class="cmc-link" href="/charts/#dominance-percentage">BTC<!-- -->: <!-- -->61.7%<!-- --> <!-- -->ETH<!-- -->: <!-- -->12.1%</a></span>,
 <span class="sc-12ja2s9-0 eALoKW"><span class="icon-Gas-Filled" style="margin-right:4px;vertical-align:middle"></span>ETH Gas<!-- -->:  <a>193<!-- --> <!-- -->Gwei<span class="icon-Chevron-down gasDownIcon___T2cjc"></span></a></span>]

In [ ]:
#Use the NY Times API to pull recent Bitcoin news

nyt_api_key = 'AHD58yFBu6t1DV7UGF2hdMCyfo2PvaHq'
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Store a search term
query = "Bitcoin"

# Search for articles published between a begin date (90 days ago) and end date (current date)

end_date = datetime.today()
begin_date = end_date - timedelta(90)

end_date = end_date.strftime('%Y%m%d')
begin_date = begin_date.strftime('%Y%m%d')


query_url = f"{url}api-key={nyt_api_key}&q={query}&begin_date={begin_date}&end_date={end_date}"

# Retrieve articles
articles = requests.get(query_url).json()

In [ ]:
# Transform API response into pandas df

headline = []
date = []
paragraph = []

articles_list = articles["response"]["docs"]

for article in articles_list:
  headline.append(article["headline"]["main"])
  date.append(article["pub_date"])
  paragraph.append(article["lead_paragraph"])

news_df = pd.DataFrame({"Date": date, "Headline": headline, "Paragraph": paragraph})
news_df['Date'] = pd.to_datetime(news_df['Date'], infer_datetime_format=True)
news_df.head(10)

**Facebook Prophet** Machine Learning

In [ ]:
# Facebook Prophet Model Setup

fb_df = df[["Date", "Close_USD"]]
fb_df = fb_df.rename(columns={"Date": "ds", "Close_USD": "y"})
fb_df.head()

In [ ]:
# Create fbProphet model and fit model 
model = Prophet()
model.fit(fb_df)

In [ ]:
# Model 90 days in the future

future_df = model.make_future_dataframe(periods=90)

In [ ]:
#Make predictions

predictions = model.predict(future_df)
predictions.tail()

In [ ]:
# Plot Predictions
model.plot(predictions)

In [ ]:
# Plot model components

model.plot_components(predictions)